
## Notebook 09: Optimización de Hiperparámetros

**Objetivo**: Comparar Grid Search + CV frente a Train-Validation Split (TVS) para maximizar el AUC del modelo de contratos de alto valor.



In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder, TrainValidationSplit
from pyspark.ml.feature import QuantileDiscretizer
from pyspark.sql.functions import col, when
import time
import json

# %%
spark = SparkSession.builder \
    .appName("SECOP_HyperparameterTuning_Logistica") \
    .master("spark://spark-master:7077") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/02/15 05:36:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable



 ### PASO 0: Preparación de Datos (Binarización)
 Replicamos la lógica para asegurar que 'label' sea 0 o 1.


In [3]:
df_raw = spark.read.parquet("/opt/spark-data/processed/secop_ml_ready.parquet")
discretizer = QuantileDiscretizer(numBuckets=4, inputCol="label", outputCol="cuartil")
df_final = discretizer.fit(df_raw).transform(df_raw) \
    .withColumn("label_bin", when(col("cuartil") == 3.0, 1.0).otherwise(0.0)) \
    .select("features", col("label_bin").alias("label"))

train, test = df_final.randomSplit([0.8, 0.2], seed=42)

# %%
# Modelo base y evaluador
lr = LogisticRegression(featuresCol="features", labelCol="label")
evaluator = BinaryClassificationEvaluator(labelCol="label", metricName="areaUnderROC")



 ## RETO 1: Diseñar el Grid de Hiperparámetros

**Pregunta de diseño**: Usamos escala logarítmica (0.01, 0.1, 1.0) porque la regularización 
 afecta el modelo en órdenes de magnitud; cambios pequeños lineales suelen ser irrelevantes.

 **Cálculo**: 3 (reg) x 2 (elastic) x 2 (iter) = **12 combinaciones**.



In [5]:
grid = ParamGridBuilder() \
    .addGrid(lr.regParam, [0.01, 0.1, 1.0]) \
    .addGrid(lr.elasticNetParam, [0.0, 1.0]) \
    .addGrid(lr.maxIter, [50, 100]) \
    .build()

print(f"Combinaciones totales: {len(grid)}")
print(f"Total modelos con K=3: {len(grid) * 3}")


Combinaciones totales: 12
Total modelos con K=3: 36


## RETO 2: Grid Search con Cross-Validation

**Pregunta**: Usamos K=3 porque el dataset es suficientemente grande para ser representativo 
 y queremos ahorrar tiempo de cómputo en el clúster.


In [7]:
cv_grid = CrossValidator(
    estimator=lr,
    estimatorParamMaps=grid,
    evaluator=evaluator,
    numFolds=3,
    seed=42
)

print("Entrenando Grid Search + CV...")
start_time = time.time()
cv_grid_model = cv_grid.fit(train)
grid_time = time.time() - start_time

best_grid_model = cv_grid_model.bestModel
auc_grid = evaluator.evaluate(best_grid_model.transform(test))

print(f"Grid Search + CV completado en {grid_time:.2f} segundos")
print(f"Mejor AUC Test (CV): {auc_grid:.4f}")

Entrenando Grid Search + CV...


26/02/15 05:36:31 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
26/02/15 05:36:31 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS
26/02/15 05:36:32 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


Grid Search + CV completado en 59.39 segundos
Mejor AUC Test (CV): 0.8262


## RETO 3: Train-Validation Split (TVS)
**Concepto**: A diferencia de CV, TVS solo entrena cada combinación 1 vez.


In [8]:
tvs = TrainValidationSplit(
    estimator=lr,
    estimatorParamMaps=grid,
    evaluator=evaluator,
    trainRatio=0.8, # 80% entrenamiento, 20% validación interna
    seed=42
)

print("Entrenando con Train-Validation Split...")
start_time = time.time()
tvs_model = tvs.fit(train)
tvs_time = time.time() - start_time

best_tvs_model = tvs_model.bestModel
auc_tvs = evaluator.evaluate(best_tvs_model.transform(test))

print(f"TVS completado en {tvs_time:.2f} segundos")
print(f"Mejor AUC Test (TVS): {auc_tvs:.4f}")


Entrenando con Train-Validation Split...


TVS completado en 17.63 segundos
Mejor AUC Test (TVS): 0.8262



 ## RETO 4: Comparar Estrategias


In [9]:
print("\n" + "="*40)
print("COMPARACIÓN DE ESTRATEGIAS")
print("="*40)
print(f"Grid Search + CV:  Tiempo: {grid_time:.1f}s | AUC: {auc_grid:.4f}")
print(f"TVS:               Tiempo: {tvs_time:.1f}s  | AUC: {auc_tvs:.4f}")

# Respuesta: TVS es notablemente más rápido. Si el AUC es similar, TVS es mejor para Big Data.



COMPARACIÓN DE ESTRATEGIAS
Grid Search + CV:  Tiempo: 59.4s | AUC: 0.8262
TVS:               Tiempo: 17.6s  | AUC: 0.8262


 ## RETO 5: Seleccionar y Guardar Modelo Final

In [10]:
# Seleccionamos el que tenga mayor AUC (mayor es mejor en clasificación)
mejor_modelo = best_grid_model if auc_grid > auc_tvs else best_tvs_model
model_path = "/opt/spark-data/processed/tuned_logistic_model"
mejor_modelo.write().overwrite().save(model_path)

# Guardar hiperparámetros
hiperparametros_optimos = {
    "regParam": float(mejor_modelo.getRegParam()),
    "elasticNetParam": float(mejor_modelo.getElasticNetParam()),
    "maxIter": int(mejor_modelo.getMaxIter()),
    "auc_test": float(max(auc_grid, auc_tvs)),
    "estrategia": "Grid Search + CV" if auc_grid > auc_tvs else "TVS"
}

with open("/opt/spark-data/processed/hiperparametros_optimos.json", "w") as f:
    json.dump(hiperparametros_optimos, f, indent=2)

print(f"Mejor modelo guardado. Estrategia ganadora: {hiperparametros_optimos['estrategia']}")


# ## RETO BONUS: Grid Más Fino
# Si el mejor regParam fue 0.01, probaremos valores muy cercanos.

fino_grid = ParamGridBuilder() \
    .addGrid(lr.regParam, [0.005, 0.01, 0.02]) \
    .addGrid(lr.elasticNetParam, [mejor_modelo.getElasticNetParam()]) \
    .build()

cv_fino = CrossValidator(estimator=lr, estimatorParamMaps=fino_grid, evaluator=evaluator, numFolds=3)
cv_fino_model = cv_fino.fit(train)
print(f"AUC Grid Fino: {evaluator.evaluate(cv_fino_model.bestModel.transform(test)):.4f}")

# ## Preguntas de Reflexión
# 1. **Grid vs Random Search**: Grid Search es exhaustivo pero lento; Random Search es mejor si tienes muchísimos parámetros y quieres explorar rápido.
# 2. **¿Por qué TVS es más rápido?**: Porque evita las rotaciones del K-Fold; solo entrena 1 vez por combinación.
# 3. **Grid demasiado grande**: El tiempo de entrenamiento crece exponencialmente, pudiendo colapsar el clúster.


Mejor modelo guardado. Estrategia ganadora: Grid Search + CV


AUC Grid Fino: 0.8262


el Train-Validation Split (TVS) es el claro ganador en términos de eficiencia. Logró exactamente el mismo AUC (0.8262) que la búsqueda exhaustiva (Grid Search + CV), pero en solo 17.6 segundos. Sin embargo, el valor de CV es un promedio de múltiples pruebas, esto hace que el modelo de Cross-Validation sea el "ganador".
el Grid Fino confirmó que el modelo ya llegó a su techo de rendimiento con un AUC de 0.8262.